In [1]:
import numpy as np
import pandas as pd
from river import optim
from river import metrics
from river.drift import EDDM
from sail.models.auto_ml.tune import SAILTuneGridSearchCV
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.models.river.linear_model import LinearRegression
from sail.models.auto_ml.auto_pipeline import SAILAutoPipeline
from sail.pipeline import SAILPipeline
from sklearn.impute import SimpleImputer
from sail.drift_detection.drift_detector import SAILDriftDetector
from sail.transfomers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../../datasets/HDWF2.csv")

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
linear_reg = LinearRegression(optimizer=optim.SGD(0.1))
random_forest = AdaptiveRandomForestRegressor()

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", "passthrough"),
]
sail_pipeline = SAILPipeline(steps=steps, scoring=metrics.R2)

#### HPO Params


In [5]:
params_grid = [
    {
        "regressor": [linear_reg],
        "regressor__l2": [0.1, 0.9],
        "regressor__intercept_init": [0.2, 0.5],
    },
    {
        "regressor": [random_forest],
        "regressor__n_models": [10, 15, 20]
    },
]

#### Create Model Instance


In [6]:
auto_pipeline = SAILAutoPipeline(
    pipeline=sail_pipeline,
    pipeline_params_grid=params_grid,
    search_method=SAILTuneGridSearchCV,
    search_method_params={
        "num_cpus_per_trial": 1,
        "max_iters": 1,
        "early_stopping": False,
        "mode": "max",
        "scoring": "r2",
        "pipeline_auto_early_stop": False,
        "keep_best_configurations": 2
    },
    search_data_size=1000,
    pipeline_strategy="PrequentialTraining",
)

[2023-07-09 23:08:25:235] - INFO - SAIL : Pipeline Strategy [PrequentialTraining] created with actions: ['DATA_COLLECTION', 'FIND_BEST_PIPELINE', 'PARTIAL_FIT_PIPELINE']


#### Start Training


In [7]:
y_preds = []
y_true = []
batch_size = 50

start = 0
for end in range(50, 2001, batch_size):

    X_train = X.iloc[start:end]
    y_train = y.iloc[start:end]

    if end > 1000: # search_data_size is 1000
        preds = auto_pipeline.predict(X_train)
        y_preds.extend(list(preds))
        y_true.extend(list(y_train))

    auto_pipeline.train(X_train, y_train)
    start = end


2023-07-09 23:08:45,364	INFO tune.py:945 -- Total run time: 16.69 seconds (16.67 seconds for the tuning loop).
[2023-07-09 23:08:50:450] - INFO - SAIL : Pipeline tuning completed. Disconnecting Ray cluster...
[2023-07-09 23:08:50:451] - INFO - SAIL : Found best params: {'regressor': AdaptiveRandomForestRegressor(n_models=20), 'regressor__n_models': 20}


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.178s, Batch Size=50, P_Score=0.739]            

>>>--------------------------------------------------------------------------------------------



SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.159s, Batch Size=50, P_Score=0.782]            


...

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.176s, Batch Size=50, P_Score=0.73]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.185s, Batch Size=50, P_Score=0.72]            


>>>--------------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.176s, Batch Size=50, P_Score=0.732]            


#### Final Score


In [8]:
auto_pipeline.progressive_score

0.7315753889477736

### Plot predictions


In [9]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds})
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()